In [11]:
#Pseudo Code:

# Set random seed for reproducibility
import numpy as np
import random
import math
			
class GPS:
    def __init__(self, apogee, total_steps, accuracy_radius):
        """
        All units are in metric - SI(standard international)
        """
        self.apogee = apogee
        self.n_steps = total_steps
        self.accuracy_radius = 5
        self.path = None

    def _parabolic_path(self):
        """
        Populates the self.path variable.
        So that you can then simulate GPS output on it.
        """
        for t, coordinate in enumerate(self.path):
            coordinate[0] = t
            coordinate[1] = (-0.001) * (t - 2000)**2 + self.apogee
            coordinate[2] = 0.0
        return self.path

    def simulate_gps(self, current_time):
        """
        Simulates coordinates from the GPS output, when the rocket is on `self.path`.
        Args:
        - current_time: needs to be between [1, total_steps]
        """
        startX=(1 + random.gauss(0, self.accuracy_radius))
        startY=(1.5 + random.gauss(0, self.accuracy_radius))
        startZ=(1.75 + random.gauss(0, self.accuracy_radius))
        
        gps_coordinates=np.ones((current_time,3,1))
        Amatrix = np.array([[1.5, 0, 0], [0, 1.75, 0], [0, 0, 1.25]])
        for i in range(current_time):
            if i ==0:
                gps_coordinates[i][0]=startX
                gps_coordinates[i][1]=startY
                gps_coordinates[i][2]=startZ
            else:
                gps_coordinates[i]=Amatrix@gps_coordinates[i-1]
                gps_coordinates[i][0] += random.gauss(0, self.accuracy_radius)
                gps_coordinates[i][1] += random.gauss(0, self.accuracy_radius)
                gps_coordinates[i][2] += random.gauss(0, self.accuracy_radius)

        return gps_coordinates		




class ParticleFilter:

    def __init__(self, Amatrix, Xvector, N = None, GPSArray = None, GPS_accuracy_radius = None):
        self.particlesArray = np.ndarray([3,N])
        self.N = 10
        self.std_dev = 3
        self.noise_std = 1
        self.old_distances = np.zeros((1,self.N))
        self.new_distances = np.zeros((1,self.N))
        self.old_particles = np.zeros((self.N,3,1))
        self.particles_curr = np.zeros((self.N,3,1), dtype=float)
        self.weights = np.zeros((self.N,1))
        self.Amatrix = np.array([[1.5, 0, 0], [0, 1.75, 0], [0, 0, 1.25]])
            
        
        self.X_GPS_curr = np.array(GPSArray[0])
        self.X_GPS_old = np.array(GPSArray[0])
        #!!!! changed particle dimensions to just 3

    def initializeParticles(self,particleLength,currGPS,std_dev):
        for i in range(particleLength):
            rX = np.random.normal(loc=currGPS[0], scale=std_dev)
            rY = np.random.normal(loc=currGPS[1], scale=std_dev)
            rZ = np.random.normal(loc=currGPS[2], scale=std_dev)
            self.particles_curr[i,0] = currGPS[0] + rX
            self.particles_curr[i,1] = currGPS[1] + rY
            self.particles_curr[i,2] = currGPS[2] + rZ
            self.weights[i] = 1/self.N

        #initialize weights to 1/n

    def calculate_theta(self, prev_pos, current_pos, particle_pos):
        # Calculate direction vectors
        prev_vector = np.array(current_pos).flatten() - np.array(prev_pos).flatten()
        particle_vector = np.array(particle_pos).flatten() - np.array(current_pos).flatten()
        
        # Normalize the vectors
        prev_vector_norm = prev_vector / np.linalg.norm(prev_vector)
        particle_vector_norm = particle_vector / np.linalg.norm(particle_vector)
        
        # Calculate the dot product
        dot_product = np.dot(prev_vector_norm, particle_vector_norm)
        
        # Clip the dot product to avoid numerical errors outside the range [-1, 1]
        dot_product = np.clip(dot_product, -1.0, 1.0)
        
        # Calculate the angle in radians and ensure it's a scalar
        theta = np.arccos(dot_product)

        return float(theta)

    def calculate_weight(self,particle_pos, prev_pos, current_pos, sigma_d = 10.0, kappa=1.0):
        d = np.linalg.norm(particle_pos-current_pos)
        #weighted by distance and angle with gaussian distribution
        distance_weight = np.exp(-d**2 / (2*sigma_d**2))
        #TODO
        theta = self.calculate_theta(prev_pos, current_pos, particle_pos)
        angle_weight = np.exp(-theta/kappa)

        return distance_weight * angle_weight
    
    def calculate_all_particle_weights(self,particles_curr, prev_pos, current_pos, sigma_d=10.0, kappa=1.0):

        for i in range(len(particles_curr)):
            self.weights[i] = self.calculate_weight(particles_curr[i], prev_pos, current_pos, 
                                sigma_d, kappa)
        self.weights += 1e-10  # Add a small constant to avoid underflow
        self.weights /= np.sum(self.weights)
        return self.weights


    def filterLowWeightParticles(self,currGPS,threshold=.95):

            cumSum = 0
            counter=0
            sortedIndeces = np.argsort(self.weights)
            for i in range(len(sortedIndeces)):
                if (1 - cumSum) <=threshold:
                    break
                cumSum += self.weights[sortedIndeces[i]]
                counter+=1

            dimensionDifference = len(self.particles_curr)-counter

            newParticleArray = np.ones((self.N,3,1))

            # We only want to copy N-counter particles (the ones we're keeping)
            for i in range(self.N - counter):
                newParticleArray[i+counter] = self.particles_curr[sortedIndeces[i+counter]]
            #from particlescurr of 0 to counter uniformally resample

            Vectoruniform_sample = np.random.uniform(low=-1, high=1, size=(counter,3,1))

            for i in range(counter):
                Vectoruniform_sample[i][0]+=currGPS[0]
                Vectoruniform_sample[i][1]+=currGPS[1]
                Vectoruniform_sample[i][2]+=currGPS[2]
                newParticleArray[self.N-i-1]=Vectoruniform_sample[i]

            weights_sum = sum(self.weights)
            newWeights = np.ones(self.N)

            # Only copy weights for the particles we're keeping (N-counter particles)
            for i in range(self.N - counter):
                newWeights[-(i+1)] = self.weights[sortedIndeces[-(i+1)]] / weights_sum
            #from newWeights of 0 to counter uniformally resample
            newWeights[:counter] = 1.0 / self.N  # or some other small constant value
# Normalize all weights to sum to 1
            newWeights = newWeights / np.sum(newWeights)
            self.weights = newWeights
            self.particles_curr=newParticleArray
            

    def progress_particles(self):
        # Motion model
        
        # Box-Muller transform for noise
        for i in range(len(self.particles_curr)):
            u1, u2 = np.random.uniform(0, 1, 2)
            r = np.sqrt(-2 * np.log(u1))
            theta = 2 * np.pi * u2
            
            # Generate two normal random variables
            z1 = r * np.cos(theta)
            z2 = r * np.sin(theta)
            z3 = np.random.normal(0, self.noise_std)  
            
            # Add noise to particle
            self.particles_curr[i] = self.Amatrix @ self.particles_curr[i] 
            self.particles_curr[i] += (np.array([z1, z2, z3]) * self.noise_std).reshape(3,1)


    def resampling(self):
        samplingThreshold = 2 * len(self.particles_curr) / 3
        print(f"Sampling threshold: {samplingThreshold}")

        # Initialize the cumulative distribution function (CDF)
        cdfArray = np.zeros(len(self.particles_curr))
        
        # Compute the cumulative sum of the weights
        for i in range(len(self.particles_curr)):
            cdfArray[i] = self.weights[i] + (cdfArray[i - 1] if i > 0 else 0)

        # Print the sum of squared weights (for debugging purpose)
        squaredWeights = np.sum(self.weights ** 2) / len(self.particles_curr)
        print(f"Squared weights: {squaredWeights}")
        
        if samplingThreshold > squaredWeights:
            rng = np.random.default_rng()  # Initialize the random number generator

            #TODO change resampled data to of self.N
            resampled_data = rng.uniform(low=0.0, high=1.0, size=int(self.N))
            print("Resampled data:", resampled_data)
            
            # Use np.searchsorted to get the indices of the resampled particles
            indices = np.searchsorted(cdfArray, resampled_data)
            print("Resampling indices:", indices)
            
            # Initialize new particles array with the correct size
            new_particles = np.zeros((len(resampled_data), 3, 1))

            # Assign resampled particles
            print("Selected resampled particles (before noise):")
            for i in range(len(resampled_data)):
                # Ensure the index is within bounds
                index = indices[i]  # Use modulo to wrap around if needed
                new_particles[i] = self.particles_curr[index]
                print(f"Resampled particle {i}: {new_particles[i]}")

            # Add noise to resampled particles
            for i in range(len(new_particles)):
                rX = np.random.normal(loc=new_particles[i][0], scale=self.std_dev)
                rY = np.random.normal(loc=new_particles[i][1], scale=self.std_dev)
                rZ = np.random.normal(loc=new_particles[i][2], scale=self.std_dev)
                
                
                # Update the particles with noise
                new_particles[i][0] += rX
                new_particles[i][1] += rY
                new_particles[i][2] += rZ
                print(f"Updated particle {i}: {new_particles[i]}")

            # Update the particles with the resampled and noise-modified particles
            self.particles_curr = new_particles
            print("Final particles after resampling and noise addition:")
            for i in range(len(self.particles_curr)):
                print(f"Particle {i}: {self.particles_curr[i]}")
def main():
    random.seed(42)
    counter = 0
    currGPS = GPS(4000,100,100)

    gps_coordinates = currGPS.simulate_gps(100)
    # print("hello", len(gps_coordinates))
    # for i in range(len(gps_coordinates)):
    #     print(gps_coordinates[i])

    Amatrix = np.array([[1.5, 0, 0], [0, 1.75, 0], [0, 0, 1.25]])
    stateVector = np.ones((3,1))
    currParticleFilter = ParticleFilter(Amatrix=Amatrix,Xvector=stateVector,N=1000,GPSArray=gps_coordinates,GPS_accuracy_radius=3)
    print("same shit")

    for i in range(20):
        print(f"we are at time step {i}")
        if i==0:
            currParticleFilter.initializeParticles(10,gps_coordinates[0],std_dev=3)
            print(f"entered initalize particles at time step {i}")
        else:
            currParticleFilter.progress_particles()
            print(f"entered progress particles at time step {i}")
            print(f"weight length {len(currParticleFilter.weights)}")
            print(f"particle length {len(currParticleFilter.particles_curr)}")
            currParticleFilter.calculate_all_particle_weights(currParticleFilter.particles_curr, gps_coordinates[i-1], gps_coordinates[i])
            print(f"entered calculateWeights particles at time step {i}")
            print(f"weight length {len(currParticleFilter.weights)}")
            print(f"particle length {len(currParticleFilter.particles_curr)}")
            currParticleFilter.resampling()
            print(f"entered reSampling particles at time step {i}")
            print(f"weight length {len(currParticleFilter.weights)}")
            print(f"particle length {len(currParticleFilter.particles_curr)}")
            currParticleFilter.filterLowWeightParticles(currGPS=gps_coordinates[i])
            print(f"entered filterParticles at time step {i}")
            print(f"weight length {len(currParticleFilter.weights)}")
            print(f"particle length {len(currParticleFilter.particles_curr)}")
#problems that may pop up in particle filter, resampling should repopulate the particles with of N length not a dependent length of the current particle array
        #initalize particles should be a uniform distribution, not a normal distribution around GPS
        #problem{1} resampling

if __name__=="__main__":
    main()

same shit
we are at time step 0
entered initalize particles at time step 0
we are at time step 1
entered progress particles at time step 1
weight length 10
particle length 10
entered calculateWeights particles at time step 1
weight length 10
particle length 10
Sampling threshold: 6.666666666666667
Squared weights: 0.01296072346622991
Resampled data: [0.60941739 0.52541493 0.45323354 0.22086552 0.26452388 0.75669891
 0.21366244 0.39628319 0.87104379 0.97795385]
Resampling indices: [5 5 4 3 3 7 3 4 7 9]
Selected resampled particles (before noise):
Resampled particle 0: [[ 6.25819063]
 [ 3.70743859]
 [-0.29713331]]
Resampled particle 1: [[ 6.25819063]
 [ 3.70743859]
 [-0.29713331]]
Resampled particle 2: [[ 6.96265338]
 [ 4.7098326 ]
 [-1.34867258]]
Resampled particle 3: [[-5.00239742]
 [ 7.27473887]
 [-1.14742071]]
Resampled particle 4: [[-5.00239742]
 [ 7.27473887]
 [-1.14742071]]
Resampled particle 5: [[ 7.50490753]
 [ 2.7155136 ]
 [-0.50933399]]
Resampled particle 6: [[-5.00239742]
 [ 

C:\Users\vijay\AppData\Local\Temp\ipykernel_11608\3389827480.py:198: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cdfArray[i] = self.weights[i] + (cdfArray[i - 1] if i > 0 else 0)
C:\Users\vijay\AppData\Local\Temp\ipykernel_11608\3389827480.py:161: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  newWeights[-(i+1)] = self.weights[sortedIndeces[-(i+1)]] / weights_sum
